## Упражнение 1

In [3]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    
async def main():
    await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

RuntimeError: This event loop is already running

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task C: Compute factorial(3)...
Task A: factorial(2) = 2
Task B: Compute factorial(3)...
Task C: Compute factorial(4)...
Task B: factorial(3) = 6
Task C: factorial(4) = 24


Оно почему-то не запусается, но если бы запускалось, то ответ был бы каким-то таким:

0: считаю 2! (итерация №1) (текущий рез = 2), после этого "2!" засыпает на 1, запускается "3!"

1: считаю 3! (итерация №1) (текущий рез = 2)

2: считаю 4! (итерация №1) (текущий рез = 2)

3: считаю 2! (итерация №2), уже посчитано, печатает результат 2! = 2

4: считаю 3! (итерация №2) (текущий рез = 6)

5: считаю 4! (итерация №2) (текущий рез = 6)

6: считаю 3! (итерация №3), уже посчитано, печатает результат 3! = 6

7: считаю 4! (итерация №3) (текущий рез = 24)

8: считаю 4! (итерация №4), уже посчитано, печатает результат 4! = 24

-----
upd, оно запустилось, моё предположение оказалось неверным, оно выполняет их в порядке, который, как пишут умные люди в интернете, заранее не определен. 

asyncio.gather() принимает корутины, заворачивает их в футуры и определяет порядок выполнения.  Порядок не предопределен тк gather пихает аргументы так, чтобы они точно ни разу не повторились

In [11]:
import sys
!{sys.executable} -m pip install aiohttp


## Упражнение 2

Из выдачи непосредственно айпишник не вырезал, потому что не получилось

In [14]:
from collections import namedtuple
import time
import asyncio
from concurrent.futures import FIRST_COMPLETED
import aiohttp

Service = namedtuple('Service', ('name', 'url', 'ip_attr'))

SERVICES = (
    'https://api.ipify.org?format=json',
    'http://ip-api.com/json'
)

async def fetch_ip(service):
    async with aiohttp.ClientSession() as session:
        async with session.get(service) as resp:
            text = await resp.text()
            return text
        


async def asynchronous():
    tasks = [fetch_ip(service) for service in SERVICES]
    while tasks:
        finished, unfinished = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
        for x in finished:
            result = x.result()
            print(result)
            if result:
                for task in unfinished:
                    task.cancel()
                await asyncio.wait(unfinished)
                return result
        tasks = unfinished

ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())

RuntimeError: This event loop is already running

{"status":"success","country":"Russia","countryCode":"RU","region":"MOW","regionName":"Moscow","city":"Moscow","zip":"129075","lat":55.7527,"lon":37.6172,"timezone":"Europe/Moscow","isp":"Net By Net Holding LLC","org":"Net By Net Holding LLC","as":"AS12714 Net By Net Holding LLC","query":"176.195.16.85"}


## Упражнение 3

In [ ]:
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor

PROXY_URL = 'socks5://xxx.xxx.xxx.xxx' # вставить здесь подходящий ip

secret_token = 'XXX'  # строка вида: 123456789:ABCDEFGHJABCDEFGHJABCDEFGHJABCDEFGHJ

bot = Bot(token=secret_token, proxy=PROXY_URL)
dp = Dispatcher(bot)


@dp.message_handler(commands=['start', 'help'])
async def send_welcome(message: types.Message):
    await message.reply("Hi!\nI'm EchoBot!\nPowered by aiogram.")


@dp.message_handler()
async def echo(message: types.Message):
    await message.reply(message.text)


if __name__ == '__main__':
    executor.start_polling(dp)